In [ ]:
# Importieren der erforderlichen Bibliotheken
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter

In [ ]:
# Dateien laden
books_df = pd.read_csv("BX_Books.csv", sep=';', encoding='latin1', on_bad_lines='skip')
users_df = pd.read_csv("BX-Users.csv", sep=';', encoding='latin1', on_bad_lines='skip')
ratings_df = pd.read_csv("BX-Book-Ratings.csv", sep=';', encoding='latin1', on_bad_lines='skip')

In [ ]:
# Zusammenführen der Daten
ratings = pd.merge(ratings_df, books_df[['ISBN', 'Book-Title']], on='ISBN', how='inner')
ratings = pd.merge(ratings, users_df[['User-ID', 'Age']], on='User-ID', how='inner')

In [ ]:
# Spaltennamen bereinigen (entfernt Leerzeichen und unbenannte Spalten)
books_df.columns = books_df.columns.str.strip()
users_df.columns = users_df.columns.str.strip()
ratings_df.columns = ratings_df.columns.str.strip()

In [ ]:
# Daten untersuchen
print("Schema:")
print(ratings_df.info())

In [ ]:
print("Anzahl der Zeilen und Spalten:")
print(f"Zeilen: {ratings_df.shape[0]}, Spalten: {ratings_df.shape[1]}")

In [ ]:
print("Erste Zeilen:")
print(ratings_df.head())

In [ ]:
# Temporäre Änderung der Anzeigeoptionen für float-Werte auf zwei Nachkommastellen
with pd.option_context('display.float_format', '{:,.2f}'.format):
    print("Descriptive Stats:")
    print(ratings_df['Book-Rating'].describe())

In [ ]:
# Anzahl der Bewertungen insgesamt
total_ratings = ratings_df.shape[0]
print(f"Anzahl Bewertungen insgesamt: {total_ratings}")

# Anzahl verschiedener Nutzer
unique_users = ratings_df['User-ID'].nunique()
print(f"Anzahl verschiedener Nutzer: {unique_users}")

# Anzahl verschiedener Bücher
unique_books = ratings_df['ISBN'].nunique()
print(f"Anzahl verschiedener Bücher: {unique_books}")

# Anzahl verschiedener Publisher
unique_publishers = books_df['Publisher'].nunique()
print(f"Anzahl verschiedener Publisher: {unique_publishers}")

In [ ]:
# Doppelte Kombinationen von userId und ISBN finden
duplicates = ratings_df.groupby(['User-ID', 'ISBN']).size()
duplicate_count = (duplicates > 1).sum()
print(f"Anzahl der doppelten Bewertungen: {duplicate_count}")

In [ ]:
# Bewertungen pro Nutzer berechnen
user_ratings_count = ratings_df.groupby('User-ID').size()

# Nutzer mit mindestens 1, 3 und 5 Bewertungen zählen
users_1 = (user_ratings_count == 1).sum()
users_min_1 = (user_ratings_count >= 1).sum()
users_min_30 = (user_ratings_count >= 30).sum()
users_min_50 = (user_ratings_count >= 50).sum()

print(f"Anzahl der Nutzer mit genau 1 Bewertung: {users_1}")
print(f"Anzahl der Nutzer mit mindestens 1 Bewertung: {users_min_1}")
print(f"Anzahl der Nutzer mit mindestens 30 Bewertungen: {users_min_30}")
print(f"Anzahl der Nutzer mit mindestens 50 Bewertungen: {users_min_50}")

In [ ]:
# Nutzer mit ausschließlich demselben Rating zählen
user_unique_ratings = ratings_df.groupby('User-ID')['Book-Rating'].nunique()
num_users_with_same_ratings = (user_unique_ratings == 1).sum()
print(f"Anzahl der Benutzer mit ausschließlich demselben Rating: {num_users_with_same_ratings}")

In [ ]:
# Bewertungen pro Buch berechnen
book_ratings_count = ratings_df.groupby('ISBN').size()

# Bücher mit mindestens 1, 3 und 5 Bewertungen zählen
books_min_0 = (book_ratings_count == 0).sum()
books_min_1 = (book_ratings_count >= 1).sum()
books_min_3 = (book_ratings_count >= 3).sum()
books_min_5 = (book_ratings_count >= 5).sum()

print(f"Anzahl der Bücher mit mindestens 0 Bewertung: {books_min_0}")
print(f"Anzahl der Bücher mit mindestens 1 Bewertung: {books_min_1}")
print(f"Anzahl der Bücher mit mindestens 3 Bewertungen: {books_min_3}")
print(f"Anzahl der Bücher mit mindestens 5 Bewertungen: {books_min_5}")

In [ ]:
# Durchschnittliche Bewertung pro Buch
avg_rating_per_book = ratings_df.groupby('ISBN')['Book-Rating'].mean()
print(avg_rating_per_book.describe())

In [ ]:
# Durchschnittliche Bewertung pro Benutzer
avg_rating_per_user = ratings_df.groupby('User-ID')['Book-Rating'].mean()
print(avg_rating_per_user.describe())

In [ ]:
# Ratings-Verteilung
rating_distribution = ratings_df['Book-Rating'].value_counts().sort_index()

# Visualisierungen
plt.figure(figsize=(10, 6))
rating_distribution.plot(kind='bar', color='skyblue')
plt.xlabel('Rating')
plt.ylabel('Anzahl der Bewertungen')

# Achsenbeschriftungen anpassen
plt.xticks(rotation=0)  # X-Achse: Werte bleiben aufrecht

# Diagramm speichern
plt.savefig("RatingsVerteilung_BookCrossing.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Verteilung der Bewertungen pro Nutzer
user_ratings_count.plot.hist(bins=50, color='lightgreen')
plt.xlabel('Anzahl der Bewertungen')
plt.ylabel('Anzahl der Nutzer')
plt.yscale('log')

# Y-Achse mit normalen Zahlen beschriften
def log_format(x, _):
    return f'{int(x):,}' if x >= 1 else ''

plt.gca().yaxis.set_major_formatter(FuncFormatter(log_format))

plt.savefig("AnzahlBewertungenNutzer_BookCrossing.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Top 10 Nutzer mit den meisten Bewertungen
top_users = user_ratings_count.nlargest(10)
top_users.plot(kind='bar', color='orange', title='Top 10 Nutzer mit den meisten Bewertungen')
plt.xlabel('Nutzer-ID')
plt.ylabel('Anzahl der Bewertungen')
plt.show()

In [ ]:
# Histogramm der durchschnittlichen Bewertungen pro Benutzer
plt.figure(figsize=(10, 6))
sns.histplot(avg_rating_per_user, bins=20, color='purple')
plt.title('Durchschnittliche Bewertung pro Benutzer')
plt.xlabel('Durchschnittliche Bewertung')
plt.ylabel('Anzahl der Benutzer')
plt.show()

In [ ]:
# Verteilung der Anzahl von Bewertungen pro Buch
book_rating_counts = ratings.groupby('ISBN').size()

plt.figure(figsize=(10, 6))
plt.hist(book_rating_counts, bins=50, color='lightgreen')
plt.title('Verteilung der Anzahl von Bewertungen pro Buch')
plt.xlabel('Anzahl der Bewertungen')
plt.ylabel('Anzahl der Bücher')

# Achse auf logarithmische Skala setzen
plt.yscale('log')

# Y-Achse mit normalen Zahlen beschriften
def log_format(x, _):
    return f'{int(x):,}' if x >= 1 else ''

plt.gca().yaxis.set_major_formatter(FuncFormatter(log_format))

plt.show()

In [ ]:
# Durchschnittliche Bewertung der Top 10 Bücher (nach Anzahl Bewertungen)
avg_rating_per_book = ratings.groupby('Book-Title')['Book-Rating'].mean()
top_books = ratings['Book-Title'].value_counts().head(10)
top_avg_ratings = avg_rating_per_book.loc[top_books.index]
plt.figure(figsize=(12, 6))
sns.barplot(x=top_avg_ratings.index, y=top_avg_ratings.values, palette='viridis')
plt.title('Durchschnittliche Bewertung der Top 10 Bücher')
plt.xlabel('Buchtitel')
plt.ylabel('Durchschnittliche Bewertung')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Top 10 Bücher mit den meisten Bewertungen
plt.figure(figsize=(12, 6))
sns.barplot(x=top_books.index, y=top_books.values, palette='viridis')
plt.title('Top 10 Bücher mit den meisten Bewertungen')
plt.xlabel('Buchtitel')
plt.ylabel('Anzahl der Bewertungen')
plt.xticks(rotation=45, ha='right')
plt.show()